In [ ]:
!./build.sh

In [ ]:
!./format.sh
!echo "=== Testing Removing Unused Import and Varialbe ==="
!autoflake --in-place --remove-unused-variables --remove-all-unused-imports get_job_status.py

In [ ]:
import os

from deploykit import (
    build_package,
    upload_package, 
    deploy_pacakge, 
    update_package_code,
    update_env_vars
)

# Test Lambda

In [ ]:
from get_job_status import handler

In [ ]:
# Testing Should Failed on status
# query = {"pid": os.environ["TEMP_PROJECT_NAME"], "status": "random_status"}
# test_event = {"queryStringParameters": query}
# handler(event=test_event, context=_)

# query = {"pid": os.environ["TEMP_PROJECT_NAME"]}
# test_event = {"queryStringParameters": query}
# handler(event=test_event, context=_)

# query = {"pid": os.environ["TEMP_PROJECT_NAME"], "status": "all"}
# test_event = {"queryStringParameters": query}
# response = handler(event=test_event, context=_)

In [ ]:
test_event = {"queryStringParameters": {"pid": os.environ["TEMP_PROJECT_NAME"], "status": "completed"}}
response = handler(event=test_event, context=_)

In [ ]:
assert response['statusCode'] == 200 

# Deploy Lambda Handler

In [ ]:
# !pip install --upgrade -r requirements.txt --target ./packages

In [ ]:
BUCKET = os.environ['BUCKET']
BUCKET_KEY = "lambdas/get-job-status.zip"
FUNCTION_NAME = "quack-tsln-get-job-status"
ENV_DYNAMODB = os.environ["DB_NAME"]

ENV_VARIABLES = {
    'Variables': {
        'BUCKET': BUCKET, 
        'DB_NAME': ENV_DYNAMODB, 
        "LAMBDA_RUNTIME_ENV": "cloud",
        "SFN_WORKFLOW_ARN": os.environ["SFN_WORKFLOW_ARN"]
    }
}
CODE_INFO = {"S3Bucket": BUCKET, "S3Key": BUCKET_KEY}
PACKAGE = BUCKET_KEY.replace("lambdas/", "")

In [ ]:
build_package(PACKAGE)

In [ ]:
upload_package(PACKAGE, BUCKET, BUCKET_KEY)

In [ ]:
DESC = "Get Job Metadata by Project ID and Status"
MAIN_PYFILE = "get_job_status"

In [ ]:
try:
    deploy_pacakge(
        function_name=FUNCTION_NAME, 
        handler=MAIN_PYFILE,
        timeout=180, 
        desc=DESC, 
        env_vars=ENV_VARIABLES,
        code_info=CODE_INFO,
        memory_size=128
    )
except Exception as e:
    print(e)
    update_package_code(function_name=FUNCTION_NAME, bucket=BUCKET, key=BUCKET_KEY)

In [ ]:
update_env_vars(function_name=FUNCTION_NAME, envvars=ENV_VARIABLES)

In [ ]:
# Create Query Parameter
# "&".join([f"{key}={value}" for key, value in test_event["queryStringParameters"].items()])